In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Leitura dos dados

In [ ]:
train_tweets = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Tweets/train_tweets.csv',index_col=0)
train_tweets_vectorized_text = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Tweets/train_tweets_vectorized_text.csv',index_col=0)


test_tweets = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Tweets/test_tweets.csv',index_col=0)
test_tweets_vectorized_text = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Tweets/test_tweets_vectorized_text.csv',index_col=0)

user_vectorized_descriptions = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Users/user_vectorized_descriptions.csv',index_col=0)
user_vectorized_profile_images = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Users/user_vectorized_profile_images.csv',index_col=0)
users = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Users/users.csv',index_col=0)

# Pré-processamento dos dados

## Redução de dimensionalidade: PCA

 Como algumas bases de dados possuem milhares de colunas, e são colunas que não possuem interpretação, é o caso perfeito para aplicação de PCA. Nesse caso foi aplicado PCA em cada base separada para dimunir o numero de features da casa de milhares para casa de dezenas.

In [ ]:
user_vectorized_descriptions_pc = pd.DataFrame(PCA(n_components=10).fit_transform(normalize(user_vectorized_descriptions)),index=user_vectorized_descriptions.index,columns=user_vectorized_descriptions.columns[0:10])
user_vectorized_profile_images_pc = pd.DataFrame(PCA(n_components=5).fit_transform(normalize(user_vectorized_profile_images)),index=user_vectorized_profile_images.index,columns=user_vectorized_profile_images.columns[0:5])

pca_tweets = PCA(n_components=25).fit(normalize(train_tweets_vectorized_text))

train_tweets_vectorized_text_pc = pd.DataFrame(pca_tweets.transform(normalize(train_tweets_vectorized_text)),index=train_tweets_vectorized_text.index,columns=train_tweets_vectorized_text.columns[0:25])
test_tweets_vectorized_text_pc = pd.DataFrame(pca_tweets.transform(normalize(test_tweets_vectorized_text)),index=test_tweets_vectorized_text.index,columns=test_tweets_vectorized_text.columns[0:25])

## Unindo as bases

Após a redução de dimensionalidade, é necessário juntar as bases para no final ter uma somente uma base de treino e uma de teste. Primeiramente as bases "user_vectorized_descriptions_pc" e "user_vectorized_profile_images_pc" foram unidas na base "users" por meio de um left join.

In [ ]:
users = users.merge(user_vectorized_descriptions_pc, on = "user_id", how = "left")
users = users.merge(user_vectorized_profile_images_pc, on = "user_id", how = "left")

Após isso a base "train_tweets_vectorized_text_pc" foi unida na "train_tweets" e a base "test_tweets_vectorized_text_pc" foi unida na "test_tweets" por meio de um left join.

In [ ]:
train = train_tweets.merge(train_tweets_vectorized_text_pc, on = "tweet_id", how = "left")
train = train.merge(users, how='left', left_on='tweet_user_id', right_on='user_id')

test = test_tweets.merge(test_tweets_vectorized_text_pc, on = "tweet_id", how = "left")
test = test.merge(users, how='left', left_on='tweet_user_id', right_on='user_id')

## Tratamento

## Removendo colunas

In [ ]:
train.tweet_topic_ids.isna().sum()

A coluna "tweet_topic_ids" foi removida por apresentar NA e por conter muitas categorias.

In [ ]:
train.drop('tweet_topic_ids', axis=1, inplace=True)
test.drop('tweet_topic_ids', axis=1, inplace=True)

## Transformando colunas booleanas em inteiras

Algumas colunas que estavam em formato booleano foram transformadas em numeric para melhor aplicabilidade nos modelos.

In [ ]:
train.tweet_has_attachment = train.tweet_has_attachment*1
test.tweet_has_attachment = test.tweet_has_attachment*1

train.user_has_url = train.user_has_url*1
test.user_has_url = test.user_has_url*1

train.user_has_location = train.user_has_location*1
test.user_has_location = test.user_has_location*1

# One hot encoding para variáveis categóricas

A coluna "tweet_attachment_class" foi desmembrada em outras 3 columas por meio de one hot enconding para melhor aplicabilidade nos modelos.

In [ ]:
dumm_tweet_attachment_class_train = pd.get_dummies(train['tweet_attachment_class'])
dumm_tweet_attachment_class_test = pd.get_dummies(test['tweet_attachment_class'])


train = pd.concat([train,dumm_tweet_attachment_class_train], axis=1)
train.drop(columns=['tweet_attachment_class'], inplace=True)

test = pd.concat([test,dumm_tweet_attachment_class_test], axis=1)
test.drop(columns=['tweet_attachment_class'], inplace=True)

## Verificando se há NA

Após todos os tratamentos foi verificado se as bases de treino e testes continham valores faltantes e foi constatado que não

In [ ]:
test.isna().sum().sum()

Parte da base de treino final.

In [ ]:
train

# Modelagem

## Criação de base de teste "gerada"

Como não consegui submeter a tempo para a competição oficial separei uma parte da base de treino para verificação da acurácia somente no final de tudo. No restante da base de treino normal utilizei Cross Validation para encontrar os melhores hiper-parametros.

In [ ]:
(train_model, test_fk) = train_test_split(train, test_size = 0.15,random_state=57)

y_train = train_model['virality']
X_train = train_model.drop(['virality','tweet_user_id'],1)

y_test_fk = test_fk['virality']
X_test_fk = test_fk.drop(['virality','tweet_user_id'],1)

Para o processo de modelogem vamos testar varios modelos e diferentes, e após isso vamos testar algumas técnicas de ensemble com esses modelos.



## Árvore de decisão

In [ ]:
#parameters = {'max_depth': [9], 'max_leaf_nodes': [87], 'min_samples_leaf': [0.008889], 'min_samples_split': [0.04737368421052632]}

#parameters = {'max_depth':list(range(2,12)), 'max_leaf_nodes': list(range(2,90))}

parameters = {'max_depth': [11], 'max_leaf_nodes': [60]}


best_params=[]
best_score = 0

i=0
# Treinamento
parameters_g = shuffle(list(ParameterGrid(parameters)))
for param in list(parameters_g):
    model_dt = tree.DecisionTreeClassifier(max_depth= param['max_depth'],
                                          max_leaf_nodes = param['max_leaf_nodes'])
    
    #model_dt.fit(X_train,y_train)
    #score = balanced_accuracy_score(y_val, model_dt.predict(X_val))

    # Validacao
    
    score_cv = cross_val_score(model_dt, X_train, y_train, cv=10,scoring='accuracy')
    score = np.mean(score_cv)
    
    
    if score > best_score:
        best_score = score
        best_params = param
        best_model_dt = model_dt

        
    i+=1
    if(i%10 == 0):
        print((i/len(list(ParameterGrid(parameters))))*100,"Best score:", best_score,"Best params:", best_params)

best_model_dt.fit(X_train, y_train)

print("Best params:", best_params)
print("Best score:", best_score)

## Florestas Aleatórias

In [ ]:
#parameters = {'max_depth':np.arange(5, 30, 2), "n_estimators":[50],  "criterion":["gini", 'entropy']}

parameters = {'criterion': ['entropy'], 'max_depth': [29], 'n_estimators': [200]}

best_params=[]
best_score = 0

i=0
# Treinamento
parameters_g = shuffle(list(ParameterGrid(parameters)))
for param in list(parameters_g):
    model_rf = RandomForestClassifier( max_depth= param['max_depth'],
                                      n_estimators= param['n_estimators'],
                                      criterion = param['criterion'])

    #model_rf.fit(X_train,y_train)
    #score = balanced_accuracy_score(y_val, model_rf.predict(X_val))

    # Validacao
    
    
    score_cv = cross_val_score(model_rf, X_train, y_train, cv=10,scoring='accuracy',n_jobs=-1)
    score = np.mean(score_cv)
    
    if score > best_score:
        best_score = score
        best_params = param
        best_model_rf = model_rf

        
    i+=1
    print((i/len(list(ParameterGrid(parameters))))*100,"Best score:", best_score,"Best params:", best_params)

best_model_rf.fit(X_train, y_train)

print("Best params:", best_params)
print("Best score:", best_score)

## XGBoost

In [ ]:
#model_xgb = XGBClassifier()

#score_cv = cross_val_score(model_xgb, X_train, y_train, cv=3,scoring='accuracy',n_jobs=-1)
#print("CV score:", np.mean(score_cv))

# Bagging (Árvore de decisão)


In [ ]:
model_bag = BaggingClassifier(n_estimators=50)
score_cv = cross_val_score(model_bag, X_train, y_train, cv=3,scoring='accuracy',n_jobs=-1)
print("CV score:", np.mean(score_cv))

model_bag.fit(X_train, y_train)

# Voting Classifier


In [ ]:
model_vot = VotingClassifier(estimators=[('rf', best_model_rf),
                                         ('bag', model_bag)],
                             voting='soft', n_jobs=-1)
score_cv = cross_val_score(model_vot, X_train, y_train, cv=3,scoring='accuracy',n_jobs=-1)
print("CV score:", np.mean(score_cv))

model_vot.fit(X_train, y_train)

# Simulação de dados Teste

In [ ]:
melhor_modelo_cv = best_model_rf

print("Test score:", accuracy_score(y_test_fk,melhor_modelo_cv.predict(X_test_fk)))

# Gerar csv submissão

In [ ]:
X_test = test.drop(['tweet_user_id'],1)

predictions = melhor_modelo_cv.predict(X_test)

predictionsDF = pd.DataFrame({"tweet_id":X_test.index, 
                                     "virality":predictions})
predictionsDF.to_csv("resposta.csv",index = False, sep = ",", decimal = ",", float_format = str)

In [ ]:
predictionsDF